## Example Config Generator 

This notebook will generate the config file for example [models/nca/](models/nca/) which implemented [Neural Cellular Automata](https://distill.pub/2020/growing-ca/). The API is decribe in [config.py](AgentTorch/config.py)

In [2]:
import numpy as np
from omegaconf import OmegaConf

import sys
sys.path.insert(0, '../../AgentTorch/')
from AgentTorch import Configurator

import types

# Create a `Configurator`

In [7]:
conf = Configurator()

# Step 1: Insert Metadata with `conf.add_metadata`

The metadata dictionary has keys as `str` and values can be `str`, `dict`, `list`, `int` or `float`

Please note that there are some **required** metdata terms that **must** always be specified. These are specified below:
1. num_episodes: Number of episodes (or epochs) for the simulation. For example: this can be the number of training epochs when optimizing simulation parameters. This is used to execute create `runner = Runner` and execute `runner.forward()`.
2. num_steps_per_episode: Number of simulation steps in each episode. For example: this can be number of days in an epidemiological simulation. Note that increasing the number of steps increases the depth of the computation graph. This is used in `runner.step(num_steps_per_episode)`
3. num_substeps_per_step: Number of distinct substeps that are being defined in each episode. This is used as a `assert` to ensure the correct number of substeps are defined with `conf.add_substeps`

Here, we are inserting other information for the size of the grid (`h`, `w`), device (`device`), optimization parameters (`learning_params`)

In [8]:
conf.add_metadata('num_episodes', 3)
conf.add_metadata('num_steps_per_episode', 20)
conf.add_metadata('num_substeps_per_step', 1)
conf.add_metadata('h', 72)
conf.add_metadata('w', 72)
conf.add_metadata('n_channels', 16)
conf.add_metadata('batch_size', 8)
conf.add_metadata('device', 'cpu')
conf.add_metadata('hidden_size', 128)
conf.add_metadata('fire_rate', 0.5)
conf.add_metadata('angle', 0.0)
conf.add_metadata('learning_params', {'lr': 2e-3, 'betas': [0.5, 0.5], 'lr_gamma': 0.9999, 'model_path': 'saved_model.pth'})


# Step 2: Create Agents and Objects using `conf.add_agents`, `conf.add_objects`

Note that you can read properties set in metadata using the `conf.get` wrapper to get variables to create agents and objects

Here, the number of automata agents is `h`*`w`

In [9]:
w, h = conf.get('simulation_metadata.w'), conf.get('simulation_metadata.h')    
automata_number = h*w
print("Automata number: ", automata_number)

automata = conf.add_agents(key="automata", number=automata_number)

Automata number:  5184


# Step 3: Create Agent Properties using conf.add_property

Each agent property has an initialization function with custom arguments. This is created using `conf.create_initializer` and `conf.create_variable`

The initialization can be done using pre-defined helper initializers in [AgentTorch_initializers.py](AgentTorch/helpers/initializer.py) or implementing custom helpers in [nca_utils.py](models/nca/substeps/utils)

In [11]:
## add properties to this agent
n_channels = conf.get('simulation_metadata.n_channels')
batch_size = conf.get('simulation_metadata.batch_size')
device = conf.get('simulation_metadata.device')

from models.nca.substeps.utils import nca_initialize_state

arguments_list = [conf.create_variable(key='n_channels', name="n_channels", learnable=False, shape=(1,), initialization_function=None, value=n_channels, dtype="int"),
                conf.create_variable(key='batch_size', name="batch_size", learnable=False, shape=(1,), initialization_function=None, value=batch_size, dtype="int"),
                conf.create_variable(key='device', name="device", learnable=False, shape=(1,), initialization_function=None, value=device, dtype="str")]

cell_state_initializer = conf.create_initializer(generator = nca_initialize_state, arguments=arguments_list)

In [12]:
automata_cell_state = conf.add_property(root='state.agents.automata', key='cell_state', name="cell_state", learnable=True, shape=(n_channels,), initialization_function=cell_state_initializer, dtype="float")

# Step 4: Create Environment Network using `conf.add_network`

Since, NCA uses a grid lattice, we can use AgentTorch helper `grid_network` from [AgentTorch/helpers/environment.py](AgentTorch/helpers/environment.py)

In [7]:
from AgentTorch.helpers.environment import grid_network
conf.add_network('evolution_network', grid_network, arguments={'shape': [w, h]})

# Step 5: Register simulation substep using `conf.add_substeps`

Please note the following:
1. The number of substeps inserted should be same as the `num_substeps_per_step` property defined in `conf.metadata`
2. Substeps are implemented in `models/nca/substeps` and registered with the `conf.add_substeps`
3. Each substep can have distinct `transition`, `observation` and `action` which are extend `SubsteTransition`, `SubstepAction` and `SubstepObservation` classes in [substep.py](AgentTorch/substep.py)

Here, NCA has a single substep to evolve the cell state `EvolveCell`. Each substep function is first define usig `create_function` and then passed to `conf.add_substep`

In [13]:
from models.nca.substeps.evolve_cell.transition import NCAEvolve
evolve_transition = conf.create_function(NCAEvolve, input_variables={'cell_state':'agents/automata/cell_state'}, output_variables=['cell_state'], fn_type="transition")

automata = conf.get('state.agents.automata')
conf.add_substep(name="Evolution", active_agents=["automata"], transition_fn=evolve_transition)

/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:18: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  Referenced from: /Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so
  Reason: image not found
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/torch_geometric/typing.py:42: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: dlopen(/Users/ayushchopra/Desktop/PROJECTS/test_birds_env/lib/python3.8/site-packages/libpyg.so, 6): Library not loaded: /usr/local/opt/python@3.10/Frameworks/Python.framework/Versions/3.10/Python
  

# Step 6: Export the Config using `conf.render`

The config file can be exported as `.yaml` file. Internally, this is an instance of [omegaconf](https://github.com/omry/omegaconf) and used by the `runner`

In [14]:
conf.render('nca_config.yaml')